In [13]:
import pandas as pd

url = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"

dados = pd.read_csv(url)

a_renomear = {
    'mileage_per_year' : 'milhas_por_ano',
    'model_year' : 'ano_modelo',
    'price' : 'preco',
    'sold' : 'vendido'
}

troca = {
    'yes' : 1,
    'no' : 0
}

dados = dados.rename(columns=a_renomear)
dados['vendido'] = dados.vendido.map(troca)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


In [14]:
from datetime import datetime

ano_atual = datetime.today().year

dados['idade_modelo'] = ano_atual - dados.ano_modelo

In [15]:
dados['km_por_ano'] = dados.milhas_por_ano * 1.60934
dados.head()

,Unnamed: 0,milhas_por_ano,ano_modelo,preco,vendido,idade_modelo,km_por_ano
0,0,21801,2000,30941.02,1,21,35085.22134
1,1,7843,1998,40557.96,1,23,12622.05362
2,2,7109,2006,89627.50,0,15,11440.79806
3,3,26823,2015,95276.14,0,6,43167.32682
4,4,7935,2014,117384.68,1,7,12770.11290


In [16]:
dados = dados.drop(columns = ['Unnamed: 0', 'milhas_por_ano', 'ano_modelo'], axis = 1)
dados.head()

,preco,vendido,idade_modelo,km_por_ano
0,30941.02,1,21,35085.22134
1,40557.96,1,23,12622.05362
2,89627.50,0,15,11440.79806
3,95276.14,0,6,43167.32682
4,117384.68,1,7,12770.11290


In [26]:
x = dados[['preco','idade_modelo','km_por_ano']]
y = dados['vendido']

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import numpy as np

SEED = 20
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

#### LINEAR SVC ####
modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("Acurácia de %.2f%%" % acuracia)
####################

Acurácia de 58.96%


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [34]:
#### Dummy Classifier ####
#### O dummy vai prever de forma burra, por exemplo chutar o valor mais
#### frequente, usa-se como guideline ou controle para o nosso
#### classificador
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
previsoes_dummy_stratified = dummy_stratified.predict(teste_x) ## o padrao do dummy é estratificado (manter as proporções)

acuracia_dummy_stratified = accuracy_score(teste_y, previsoes_dummy_stratified) * 100
print("Acurácia do Dummy Stratified de %.2f%%" % acuracia_dummy_stratified)

dummy_mostfrequent = DummyClassifier(strategy='most_frequent')
dummy_mostfrequent.fit(treino_x, treino_y)
previsoes_dummy_mostfrequent = dummy_mostfrequent.predict(teste_x)

acuracia_dummy_mostfrequent = accuracy_score(teste_y, previsoes_dummy_mostfrequent) * 100
print("Acurácia do Dummy Most Frequent de %.2f%%" % acuracia_dummy_mostfrequent)
dummy_mostfrequent.score(teste_x, teste_y) ## em vez de usar o accuracy score, usa o metodo score dos classifiers

Acurácia do Dummy Stratified de 50.72%
Acurácia do Dummy Most Frequent de 58.00%


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


0.58

In [40]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

np.random.seed(20)
raw_treino_x, raw_teste_x, raw_treino_y, raw_teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
print("Acurária SVC: %.2f%%" % (modelo.score(teste_x, teste_y) * 100))

Acurária SVC: 75.92%
